# Session 8 - Language modelling with RNNs (Text Generation)

In [1]:
# data processing tools
import string, os 
import pandas as pd
import numpy as np
np.random.seed(42)

# keras module for building LSTM 
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.utils as ku 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# surpress warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(
    action = 'ignore', 
    category = FutureWarning)

2023-03-30 14:43:31.233430: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 14:43:31.281666: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 14:43:31.282812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 14:43:32.082278: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Some helper functions

In [2]:
def clean_text(txt):
    # loop through string character by character. check if character is part
    # of the string.punctuation vector and if yes then remove it. convert 
    # everything else to lowercase. concatenate to form new string
    # resulting txt string will contain the original string with all 
    # punctuation marks removed and all characters converted to lowercase
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    # convert text to bytes with UTF-8 encoding, then take resulting output
    # and translate it back into strings while keeping all ASCII-characters
    # and removing non-ASCII characters. great if original string contains 
    # characters that cannot be processed by certain systems or software
    txt = txt.encode("utf8").decode(
        "ascii",
        'ignore')
    return txt 

def get_sequence_of_tokens(tokenizer, corpus):
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        # convert each line into a sequence of integers
        token_list = tokenizer.texts_to_sequences([line])[0]
        # for each token, find the token that comes after it
        # add to list
        # (thereby you have a nice list with input words and words that can
        # come after them)
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

def generate_padded_sequences(input_sequences):
    # get the length of the longest sequence
    max_sequence_len = max([len(x) for x in input_sequences])
    # make every sequence the length of the longest on
    input_sequences = np.array(pad_sequences(input_sequences, 
                                            maxlen=max_sequence_len, 
                                            padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, 
                            num_classes=total_words)
    return predictors, label, max_sequence_len

def create_model(max_sequence_len, total_words):
    # max_sequence_len: an integer representing the maximum length of a text sequence
    # total_words: an integer representing the total number of unique words in the 
    # vocabulary of the text corpus
    input_len = max_sequence_len - 1
    model = Sequential() # model takes one word after the other sequentially
    
    # Add Input Embedding Layer
    # creates dense vector representation for each input word
    model.add(
        Embedding(
            total_words, # the total number of unique words in the vocabulary
            10, # the size of the vector space in which the words will be embedded
            input_length = input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(
        LSTM(
            100)) # number of memory cells in the layer
    # prevent overfitting by randomly dropping out some of the connections between 
    # the LSTM cells
    model.add(
        Dropout(
            0.1)) # dropout rate
    
    # Add Output Layer
    # generate a probability distribution over the vocabulary of possible next words 
    # in the sequence
    model.add(
        Dense(
            total_words, # the total number of unique words in the vocabulary
            activation = 'softmax'))

    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'adam')
    
    return model

def generate_text(seed_text, next_words, model, max_sequence_len):
    # seed_text: a string representing the starting text for text generation
    # next_words: an integer representing the number of words to generate after 
    # the seed text
    # model: a trained Keras neural network model that will be used for text 
    # generation
    # max_sequence_len: an integer representing the maximum length of the input 
    # sequence that the model was trained on
    for _ in range(next_words):
        # convert seed_text to tokens
        token_list = tokenizer.texts_to_sequences([
            seed_text])[0]
        # pad the sequence with zeros to match the length of the input sequences 
        # that the model was trained on
        token_list = pad_sequences([
            token_list],
            maxlen = max_sequence_len-1,
            padding = 'pre') # add zeros before the sequence
        # determine the index of the word with the highest predicted probability 
        # in the output vocabulary
        predicted = np.argmax(
            model.predict(
                token_list),
                axis = 1)
        
        output_word = ""
        # look up the actual word corresponding to the predicted index in the 
        # tokenizer's word index, then appends this word to the seed_text variable
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

## Load the data

In [3]:
data_dir = os.path.join(
    "..", 
    "..", 
    "..", 
    "431868", 
    "news_data")

We're then going to load the data one at a time and append *only* the headlines to our list of data.

In [5]:
all_headlines = []
for filename in os.listdir(data_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(data_dir + "/" + filename)
        all_headlines.extend(list(article_df["headline"].values))#keep headline

We then clean up a little bit and see how many data points we have.

In [6]:
# remove "Unknown" headlines
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

8603

We call our ```clean_text()``` function and then inspect the first 10 texts.

In [7]:
corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal',
 'an antidote to europes populism',
 'the cost of a speech',
 'degradation of the language',
 'on the power of being awful',
 'trump garbles pitch on a revised health bill']

## Tokenize

We're then going to tokenize our data, using the ```Tokenizer()``` class from ```TensorFlow```, about which you can read more [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer).

We then use the ```get_sequence_of_tokens()``` function we defined above, which turns every text into a sequence of tokens based on the vocabulary from the tokenizer.

In [8]:
# note: tensorflow also has a cleaning function that can be used 
# instead of the predefined function

tokenizer = Tokenizer()
## tokenization
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [13]:
total_words

11265

In [14]:
inp_sequences = get_sequence_of_tokens(tokenizer, corpus)
inp_sequences[:10]
# The words in the headlines have been replaced by integers.
# For each headline, multiple n-grams have been generated.
# It makes more data, shows the way a sentence 'grows', and
# teaches the model how to deal with input sentences of different 
# lengths.

[[46, 1601],
 [46, 1601, 1],
 [46, 1601, 1, 1951],
 [46, 1601, 1, 1951, 120],
 [122, 331],
 [122, 331, 1952],
 [122, 331, 1952, 2],
 [122, 331, 1952, 2, 125],
 [122, 331, 1952, 2, 125, 2484],
 [122, 331, 1952, 2, 125, 2484, 812]]

We then want to *pad* our input sequences to make them all the same length.

In [15]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

## Create model

We then use the ```create_model()``` function created above to initialize a model, telling the model the length of sequences and the total size of the vocabulary.

In [16]:
model = create_model(
    max_sequence_len, 
    total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            112650    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 11265)             1137765   
                                                                 
Total params: 1,294,815
Trainable params: 1,294,815
Non-trainable params: 0
_________________________________________________________________


2023-03-30 15:19:26.516326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-30 15:19:26.517724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-30 15:19:26.519860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model training is exactly the same as last week, but instead of document labels, we're fitting the model to predict next word.

*NB!* This will take some time to train! It took me 35 minutes on UCloud 32xCPU.

In [17]:
history = model.fit(
    predictors,
    label, 
    epochs = 100,
    batch_size = 128,
    verbose = 1)

Epoch 1/100


2023-03-30 15:20:21.860259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-30 15:20:21.861666: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-30 15:20:21.863243: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

405/405 [==============================] - 23s 51ms/step - loss: 7.8888
Epoch 2/100
405/405 [==============================] - 20s 49ms/step - loss: 7.4823
Epoch 3/100
405/405 [==============================] - 19s 48ms/step - loss: 7.3965
Epoch 4/100
405/405 [==============================] - 20s 48ms/step - loss: 7.3074
Epoch 5/100
405/405 [==============================] - 20s 48ms/step - loss: 7.2040
Epoch 6/100
405/405 [==============================] - 20s 49ms/step - loss: 7.0989
Epoch 7/100
405/405 [==============================] - 20s 49ms/step - loss: 6.9847
Epoch 8/100
405/405 [==============================] - 20s 48ms/step - loss: 6.8567
Epoch 9/100
405/405 [==============================] - 20s 49ms/step - loss: 6.7186
Epoch 10/100
405/405 [==============================] - 20s 48ms/step - loss: 6.5817
Epoch 11/100
405/405 [==============================] - 20s 49ms/step - loss: 6.4487
Epoch 12/100
405/405 [==============================] - 20s 48ms/step - loss: 6.3247
E

When the model has trained, we can then use this to generate *new text*.

In [18]:
print(generate_text("danish", 5, model, max_sequence_len))

1/1 [==============================] - 0s 329ms/step


2023-03-30 15:56:32.418588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-30 15:56:32.420524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-30 15:56:32.421618: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 28ms/step
Danish Inventor Is Found Guilty In


## Using pre-trained word embeddings

Instead of having the embedding layer as a trainable parameter, we can instead using a *pretrained word embedding* model like ```word2vec```.

In the following examples, we're using [GloVe embeddings](https://nlp.stanford.edu/projects/glove/). These are trained a little differently from ```word2vec``` but they behave in the same way.

In [19]:
path_to_glove_file = os.path.join("path/to/glove/vectors")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/glove/vectors'

We can define some variables that we're going to use later.

With hits and misses, we're counting how many words in the corpus vocabulary have a corresponding GloVe embedding; misses are the words which appear in our vocabulary but which do not have a GloVe embedding.

In [ ]:
num_tokens = total_words
embedding_dim = 100
hits = 0
misses = 0

In [ ]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer - notice that this is different
    model.add(Embedding(
            total_words,
            embedding_dim,
            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
            trainable=False,
            input_length=input_len)
    )
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(500))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    
    return model

In [ ]:
model = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)

In [ ]:
print (generate_text("china", 30, model, max_sequence_len))